In [1]:
import pandas as pd
from pymongo import MongoClient

from IPython.core.display import display, HTML
[display(HTML("<style>.container { width:100% !important; }</style>"))];

In [2]:
with MongoClient('mongodb://localhost:27017/') as client:
    database = client["dwh"]
    
    country_code_collection = database['country_codes']
    country_code_collection.drop()
    country_code_collection.insert_many([
        {data['country']: {'who': data['code_who'], 'noc': data['code_noc']}}
        for _, data in pd.read_csv('data/country_codes.csv').iterrows()
    ])
    
    who_ppp_collection = database['who_ppp_2017']
    who_ppp_collection.drop()
    data = []
    who_ppp_pc, who_ppp_abs = pd.read_csv('data/ppp_2017_abs.csv'), pd.read_csv('data/ppp_2017_pc.csv')
    for _, row in who_ppp_pc.iterrows():
        data.append({row['Country Code']: {'pc': float(row['Value']), 'abs': float(who_ppp_abs[who_ppp_abs['Country Code'] == row['Country Code']]['Value'])}})
    who_ppp_collection.insert_many(data)

    medal_wins_collection = database['medal_wins']
    medal_wins_collection.drop()
    wins_by_noc = {}
    for _, row in pd.read_csv('data/medal_wins.csv').iterrows():
        wins_by_noc[noc] = wins_by_noc.get((noc := str(row['NOC'])), []) + [row[['Name', 'Year', 'Sport', 'Medal']]]
    medal_wins_collection.insert_many([{k: list({'name': x[0], 'year': x[1], 'sport': x[2], 'medal': x[3]} for x in v)} for k, v in wins_by_noc.items()])